In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import time, json, datetime 
from tqdm import tqdm

import numpy as np 
import pandas as pd 
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [12]:
columns = ['label', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11',
       'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
       'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
       'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']
data = pd.read_csv('./dac_sample.txt',sep='\t',header=None, names=columns)
print(data.shape)
data.head()

(100000, 40)


,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [13]:
# data = pd.read_csv("criteo_sample_50w.csv")

dense_features = [f for f in data.columns.tolist() if f[0] == "I"]
sparse_features = [f for f in data.columns.tolist() if f[0] == "C"]

data[sparse_features] = data[sparse_features].fillna('-10086', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

In [15]:
## 类别特征labelencoder
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

## 数值特征标准化
for feat in dense_features:
    mean = data[feat].mean()
    std = data[feat].std()
    data[feat] = (data[feat] - mean) / (std + 1e-12)   # 防止除零

print(data.shape)
data.head()

(100000, 40)


,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,-0.136628,-0.278599,-0.057666,-0.649408,-0.239004,-0.307323,0.007756,-0.248508,0.219409,...,9,2422,150,3,1163,0,2,9570,41,5604
1,0,-0.011844,-0.281089,0.022764,-0.551940,-0.258904,-0.295105,-0.195436,-0.248508,-0.410853,...,0,1727,150,1,14512,0,2,3233,41,4215
2,0,-0.011844,-0.281089,-0.065915,0.715132,-0.248565,-0.047694,-0.164176,-0.248508,0.447300,...,6,510,0,0,34644,6,2,2818,0,0
3,0,-0.261411,1.942945,-0.067977,-0.649408,-0.192208,-0.319541,-0.226697,-0.291501,-0.425097,...,1,1170,0,0,15997,0,2,6971,0,0
4,0,0.112939,-0.283580,-0.067977,-0.649408,-0.260458,-0.319541,-0.179806,-0.291501,-0.425097,...,1,390,0,0,5101,0,1,8663,0,0


In [18]:
train, valid = train_test_split(data, test_size=0.2, random_state=2020)
print(train.shape, valid.shape)

train_dataset = Data.TensorDataset(torch.LongTensor(train[sparse_features].values), 
                                   torch.FloatTensor(train[dense_features].values),
                                   torch.FloatTensor(train['label'].values),)

train_loader = Data.DataLoader(dataset=train_dataset, batch_size=2048, shuffle=True)

valid_dataset = Data.TensorDataset(torch.LongTensor(valid[sparse_features].values), 
                                   torch.FloatTensor(valid[dense_features].values),
                                   torch.FloatTensor(valid['label'].values),)
valid_loader = Data.DataLoader(dataset=valid_dataset, batch_size=4096, shuffle=False)


# 打印模型参数
def get_parameter_number(model):
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}
print(get_parameter_number(model))

# 定义日志（data文件夹下，同级目录新建一个data文件夹）
def write_log(w):
    file_name = 'data/' + datetime.date.today().strftime('%m%d')+"_{}.log".format("deepfm")
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}".format(t0, w)
    print(info)
    with open(file_name, 'a') as f: 
        f.write(info + '\n') 

(80000, 40) (20000, 40)
{'Total': 1269936, 'Trainable': 1269936}


In [53]:
class DeepFM(nn.Module):
    def __init__(self, cate_fea_nuniqs, nume_fea_size=0, emb_size=8, 
                 hid_dims=[256, 128], num_classes=1, dropout=[0.2, 0.2]): 
        """
        cate_fea_nuniqs: 类别特征的唯一值个数列表，也就是每个类别特征的vocab_size所组成的列表
        nume_fea_size: 数值特征的个数，该模型会考虑到输入全为类别型，即没有数值特征的情况 
        """
        super().__init__()
        self.cate_fea_size = len(cate_fea_nuniqs)
        self.nume_fea_size = nume_fea_size
        
        """FM部分"""
        # 一阶
        if self.nume_fea_size != 0:
            self.fm_1st_order_dense = nn.Linear(self.nume_fea_size, 1)  # 数值特征的一阶表示
        self.fm_1st_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, 1) for voc_size in cate_fea_nuniqs])  # 类别特征的一阶表示
        
        # 二阶
        self.fm_2nd_order_sparse_emb = nn.ModuleList([
            nn.Embedding(voc_size, emb_size) for voc_size in cate_fea_nuniqs])  # 类别特征的二阶表示
        
        """DNN部分"""
        # self.all_dims = [self.cate_fea_size * emb_size] + hid_dims
        self.all_dims = [(self.cate_fea_size * emb_size)+self.nume_fea_size] + hid_dims
        self.dense_linear = nn.Linear(self.nume_fea_size, self.cate_fea_size * emb_size)  # 数值特征的维度变换到FM输出维度一致
        self.relu = nn.ReLU()
        # for DNN 
        for i in range(1, len(self.all_dims)):
            setattr(self, 'linear_'+str(i), nn.Linear(self.all_dims[i-1], self.all_dims[i]))
            setattr(self, 'batchNorm_' + str(i), nn.BatchNorm1d(self.all_dims[i]))
            setattr(self, 'activation_' + str(i), nn.ReLU())
            setattr(self, 'dropout_'+str(i), nn.Dropout(dropout[i-1]))
        # for output 
        self.dnn_linear = nn.Linear(hid_dims[-1], num_classes)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, X_sparse, X_dense=None):
        """
        X_sparse: 类别型特征输入  [bs, cate_fea_size]
        X_dense: 数值型特征输入（可能没有）  [bs, dense_fea_size]
        """
        
        """FM 一阶部分"""
        fm_1st_sparse_res = [emb(X_sparse[:, i].unsqueeze(1)).view(-1, 1) 
                             for i, emb in enumerate(self.fm_1st_order_sparse_emb)]
        fm_1st_sparse_res = torch.cat(fm_1st_sparse_res, dim=1)  # [bs, cate_fea_size]
        fm_1st_sparse_res = torch.sum(fm_1st_sparse_res, 1,  keepdim=True)  # [bs, 1]
        
        if X_dense is not None:
            fm_1st_dense_res = self.fm_1st_order_dense(X_dense) 
            fm_1st_part = fm_1st_sparse_res + fm_1st_dense_res
        else:
            fm_1st_part = fm_1st_sparse_res   # [bs, 1]
        
        """FM 二阶部分"""
        fm_2nd_order_res = [emb(X_sparse[:, i].unsqueeze(1)) for i, emb in enumerate(self.fm_2nd_order_sparse_emb)]
        fm_2nd_concat_1d = torch.cat(fm_2nd_order_res, dim=1)  # [bs, n, emb_size]  n为类别型特征个数(cate_fea_size)
        
        # 先求和再平方
        sum_embed = torch.sum(fm_2nd_concat_1d, 1)  # [bs, emb_size]
        square_sum_embed = sum_embed * sum_embed    # [bs, emb_size]
        # 先平方再求和
        square_embed = fm_2nd_concat_1d * fm_2nd_concat_1d  # [bs, n, emb_size]
        sum_square_embed = torch.sum(square_embed, 1)  # [bs, emb_size]
        # 相减除以2 
        sub = square_sum_embed - sum_square_embed  
        sub = sub * 0.5   # [bs, emb_size]
        
        fm_2nd_part = torch.sum(sub, 1, keepdim=True)   # [bs, 1]
        
        """DNN部分"""
        dnn_out = torch.flatten(fm_2nd_concat_1d, 1)   # [bs, n * emb_size]
        
        if X_dense is not None:
            dense_out = self.relu(self.dense_linear(X_dense))   # [bs, n * emb_size]
            # dnn_out = dnn_out + dense_out   # [bs, n * emb_size]
            dnn_out = torch.cat([dnn_out, X_dense], dim=1) # [bs, n * emb_size + dense_fea_szie]
        
        for i in range(1, len(self.all_dims)):
            dnn_out = getattr(self, 'linear_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'batchNorm_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'activation_' + str(i))(dnn_out)
            dnn_out = getattr(self, 'dropout_' + str(i))(dnn_out)
        
        dnn_out = self.dnn_linear(dnn_out)   # [bs, 1]
        out = fm_1st_part + fm_2nd_part + dnn_out   # [bs, 1]
        out = self.sigmoid(out)
        return out

In [ ]:
"""一些可以调整的地方
1.dnn层输入特征
    - dnn_out = dnn_out + dense_out   # [bs, n * emb_size]
    - dnn_out = torch.cat([dnn_out, X_dense], dim=1) # [bs, n * emb_size + dense_fea_szie]
    
2.sigmoid层输入
    - element-wise: fm_1st_part + fm_2nd_part + dnn_out   # [bs, 1]
    - concat: 修改预测函数
    def binary_prediction_layer(final_logit: torch.Tensor):
        # 将fm+dnn的输出concat后加一层线性层
        outputs = nn.Linear(final_logit.size(1), 1)(final_logit)
        print(outputs)
        outputs = torch.sigmoid(outputs)
        return outputs
"""

In [54]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
cate_fea_nuniqs = [data[f].nunique() for f in sparse_features]
model = DeepFM(cate_fea_nuniqs, nume_fea_size=len(dense_features))
model.to(device)
loss_fcn = nn.BCELoss()  # Loss函数
loss_fcn = loss_fcn.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)

cpu


In [55]:
def train_and_eval(model, train_loader, valid_loader, epochs, device):
    best_auc = 0.0
    for _ in range(epochs):
        """训练部分"""
        model.train()
        print("Current lr : {}".format(optimizer.state_dict()['param_groups'][0]['lr']))
        write_log('Epoch: {}'.format(_ + 1))
        train_loss_sum = 0.0
        start_time = time.time()
        for idx, x in enumerate(train_loader):
            cate_fea, nume_fea, label = x[0], x[1], x[2]
            cate_fea, nume_fea, label = cate_fea.to(device), nume_fea.to(device), label.float().to(device)
            pred = model(cate_fea, nume_fea).view(-1)
            loss = loss_fcn(pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss_sum += loss.cpu().item()
            if (idx+1) % 50 == 0 or (idx + 1) == len(train_loader):
                write_log("Epoch {:04d} | Step {:04d} / {} | Loss {:.4f} | Time {:.4f}".format(
                          _+1, idx+1, len(train_loader), train_loss_sum/(idx+1), time.time() - start_time))
        scheduler.step()
        """推断部分"""
        model.eval()
        with torch.no_grad():
            valid_labels, valid_preds = [], []
            for idx, x in tqdm(enumerate(valid_loader)):
                cate_fea, nume_fea, label = x[0], x[1], x[2]
                cate_fea, nume_fea = cate_fea.to(device), nume_fea.to(device)
                pred = model(cate_fea, nume_fea).reshape(-1).data.cpu().numpy().tolist()
                valid_preds.extend(pred)
                valid_labels.extend(label.cpu().numpy().tolist())
        cur_auc = roc_auc_score(valid_labels, valid_preds)
        if cur_auc > best_auc:
            best_auc = cur_auc
            torch.save(model.state_dict(), "data/deepfm_best.pth")
        write_log('Current AUC: %.6f, Best AUC: %.6f\n' % (cur_auc, best_auc))

In [56]:
train_and_eval(model, train_loader, valid_loader, 5, device)

Current lr : 0.005
10:36:09 : Epoch: 1
10:36:13 : Epoch 0001 | Step 0040 / 40 | Loss 21.5357 | Time 4.6699


5it [00:00, 10.88it/s]


10:36:14 : Current AUC: 0.590020, Best AUC: 0.590020

Current lr : 0.004
10:36:14 : Epoch: 2
10:36:19 : Epoch 0002 | Step 0040 / 40 | Loss 13.4166 | Time 4.6223


5it [00:00, 10.93it/s]


10:36:19 : Current AUC: 0.648386, Best AUC: 0.648386

Current lr : 0.0032
10:36:19 : Epoch: 3
10:36:24 : Epoch 0003 | Step 0040 / 40 | Loss 11.3209 | Time 4.5787


...

10:36:24 : Current AUC: 0.667182, Best AUC: 0.667182

Current lr : 0.00256
10:36:24 : Epoch: 4
10:36:29 : Epoch 0004 | Step 0040 / 40 | Loss 9.6106 | Time 5.0222


5it [00:00, 23.16it/s]


10:36:30 : Current AUC: 0.666291, Best AUC: 0.667182

Current lr : 0.0020480000000000003
10:36:30 : Epoch: 5
10:36:35 : Epoch 0005 | Step 0040 / 40 | Loss 7.7804 | Time 4.9468


5it [00:00, 23.76it/s]


10:36:35 : Current AUC: 0.675655, Best AUC: 0.675655

